In [47]:
# NOTE need to change {event_year}-TEST

import pandas as pd
import numpy as np
import random
import os
from docx import Document
from docx.shared import Inches, Pt

# implementing random seed for control
sd = 42
np.random.seed(sd)

# school names for consistency 
schools = ['Oakland Middle School',
    'Siegel Middle School',
    'Whitworth-Buchanan Middle School',
    'Christiana Middle School',
    'Smyrna Middle School',
    'Stewarts Creek Middle School',
    'Rockvale Middle School',
    'Rocky Fork Middle School',
    'Blackman Middle School',
    'Thurman Francis Arts Academy',
    'Rock Springs Middle School',
    'LaVergne Middle School'
]

# get event year
try:
    event_year = int(input('Enter 4-digit year of YouScience Event\n'))
except ValueError:
    event_year = 2022
print(f"Running for YS Career Fair {event_year}")

# preparing directories
for school in schools:
    path = f'../YouScienceData/Schedules/{event_year}-TEST/{school}'
    schedule_path = f'../YouScienceData/Schedules/{event_year}-TEST/{school}/Student_Schedules'
    if not os.path.exists(path):
        os.makedirs(path)
    if not os.path.exists(schedule_path):
        os.makedirs(schedule_path)


Running for YS Career Fair 2022


In [51]:
# Date of YouScience Career Fair
date = '8 February 2023'

def build_student_schedules(school_df, rooms_df):
    # iterate through students
    for i in range(len(school_df)):
        student_case = school_df.iloc[i]
        # get (1) student
        student_name = ' '.join([str(student_case['First_Name']), str(student_case['Last_Name'])])
        # a weird case that showed up once
        if student_name == 'JORDIN/ALBUS MCCLANAHAN':
            student_name = 'JORDIN MCCLANAHAN'
        elif "Unassigned" in student_name:
            num = student_name[-1]
            student_name = f'{num} {student_name[:10]}'
        
        # initialize schedule document & header format
        schedule_doc = Document()
        section = schedule_doc.sections[0]
        header = section.header
        head_para = header.paragraphs[0]
        r = head_para.add_run()
        instr_str = 'We are excited about the opportunities you will have today. Ask plenty of questions & take notes. Fill in the exit ticket using the QR code at the end of your last session.'
        r.text = f"{student_case.School}\tRutherford County School's Career Pathway Fair\t{date}\nWelcome: {student_name}\t\t"
        r.add_picture("../Deliverables/Templates/QR_Career_Fair_2023.png", width=Inches(1.25))
        # schedule_doc.add_heading(f'Welcome {student_name}', level = 1)
        p = schedule_doc.add_paragraph()
        r = p.add_run()
        r.text = f"{instr_str}"
        #r.add_picture("../Deliverables/Templates/QR_Career_Fair_2023.png", width=Inches(1.25))

        # build document body
        for block in ['B1','B2','B3','B4']: 
            # get pathway's info
            # (3a)
            pathway = student_case[block]
            POS_info = rooms_df.loc[rooms_df.POS == pathway]         
            POS_room_no = POS_info.Room.values[0]

            # structure document
            schedule_doc.add_heading(f'Program of Study: {pathway}, Room: {POS_room_no}', level=3)
            schedule_doc.add_picture(f"../Deliverables/Templates/template_s{block[1]}.png", width=Inches(6))
            

        school_init = ''.join([x[0] for x in student_case.School.split()])
        name_for_path = '_'.join([school_init, student_name])
        doc_path = f'../YouScienceData/Schedules/{event_year}-TEST/{school}/Student_Schedules/{name_for_path}.docx'
        schedule_doc.save(doc_path)

    return i+1

def Print_Schedules(school=None, date=date):

    if school == None:
        return 0

    # load in appropriate dfs: complete schedules & room assignments
    school_path = f'../YouScienceData/Schedules/{event_year}-TEST/{school}/{school}_all_students.csv'
    rooms_path = f'../YouScienceData/Schedules/{event_year}-TEST/{school}/room_assignments_by_demand.csv'
    school_df = pd.read_csv(school_path).drop('Unnamed: 0', axis=1)
    rooms_df = pd.read_csv(rooms_path).drop('Unnamed: 0', axis=1)

    # execute schedule building
    n = build_student_schedules(school_df=school_df, rooms_df=rooms_df)
    print(f'{school} scheduls created: {n} of {len(school_df)}.')
    return 

for school in schools:
    if school in ['Thurman Francis Arts Academy', 'Christiana Middle School']:
        print(f'--{school} omitted, missing data.')
        continue
    
    Print_Schedules(school=school, date=date)
    

Oakland Middle School scheduls created: 440 of 440.
Siegel Middle School scheduls created: 455 of 455.
Whitworth-Buchanan Middle School scheduls created: 342 of 342.
Christiana Middle School omitted, missing data.
Smyrna Middle School scheduls created: 372 of 372.
Stewarts Creek Middle School scheduls created: 401 of 401.
Rockvale Middle School scheduls created: 563 of 563.
Rocky Fork Middle School scheduls created: 293 of 293.
Blackman Middle School scheduls created: 642 of 642.
Thurman Francis Arts Academy omitted, missing data.
Rock Springs Middle School scheduls created: 353 of 353.
LaVergne Middle School scheduls created: 428 of 428.


# Troubleshooting

Note: python-docx is still young. It is a pain, but I didn't feel like trying to figure out how to get microsoft to play nicely to populate templates. 

In [7]:
# during test, feature of the school name, has been updated that next run will have the feature
# labeled "School" rather than the literal school. 
school = 'Blackman Middle School'
p = f'../YouScienceData/Schedules/2022-TEST/{school}/room_assignments_by_demand.csv'
X = pd.read_csv(p).drop('Unnamed: 0', axis=1)
X.head()

,Room,Capacity,POS,Blackman Middle School
0,A109,60,Sport & Human Performance,Blackman Middle School
1,A111,60,Veterinary & Animal Science,Blackman Middle School
2,A209,60,Horticulture Sciences,Blackman Middle School
3,A211,60,Cybersecurity,Blackman Middle School
4,A101,35,Coding,Blackman Middle School


In [9]:
p = f'../YouScienceData/Schedules/{event_year}-TEST/{school}/{school}_all_students.csv'
df = pd.read_csv(p).drop('Unnamed: 0', axis=1)
df.head(10)

,id,School,First_Name,Last_Name,Email,B1,B2,B3,B4,CY Teacher
0,Empty 0,Blackman Middle School,Unassigned,Open 1,Open 1,Culinary Arts,Banking & Finance,Culinary Arts,Cybersecurity,NaN
1,Empty 1,Blackman Middle School,Unassigned,Open 2,Open 2,Veterinary & Animal Science,Leadership in Government,Culinary Arts,STEM Engineering,NaN
2,Empty 2,Blackman Middle School,Unassigned,Open 3,Open 3,Banking & Finance,Culinary Arts,Culinary Arts,STEM Engineering,NaN
3,Empty 3,Blackman Middle School,Unassigned,Open 4,Open 4,Leadership in Government,Business Management,Banking & Finance,Cybersecurity,NaN
4,Empty 4,Blackman Middle School,Unassigned,Open 5,Open 5,Culinary Arts,Business Management,Leadership in Government,Business Management,NaN
5,1429415,Blackman Middle School,ALLEN,BERKHIMER,aberkhimer000@student.rcschools.net,Marketing Management,Banking & Finance,Horticulture Sciences,Cybersecurity,"0, Missing"
6,1428797,Blackman Middle School,CALEB,BESS,cbess6875@student.rcschools.net,Veterinary & Animal Science,Sport & Human Performance,STEM Engineering,Cybersecurity,"0, Missing"
7,1429317,Blackman Middle School,JACE,BUCHHOLZ,jbuchholz7367@student.rcschools.net,Sport & Human Performance,Veterinary & Animal Science,Horticulture Sciences,Criminal Justice & Correction Services,"0, Missing"
8,1429325,Blackman Middle School,Patrick,Brockley,pbrockley000@student.rcschools.net,Cybersecurity,Criminal Justice & Correction Services,Horticulture Sciences,Coding,"0, Missing"
9,1430217,Blackman Middle School,ROLAND,CRAWLEY,rcrawley000@student.rcschools.netassssssssssss...,STEM Engineering,Veterinary & Animal Science,Sport & Human Performance,Cybersecurity,"0, Missing"


In [12]:
student_case = df.iloc[9]
' '.join([student_case['First_Name'], student_case['Last_Name']])

'ROLAND CRAWLEY'

In [20]:
X.loc[X.POS == student_case.B1].Room.values[0]


'A110'

In [22]:
''.join([x[0] for x in student_case.School.split()])

'BMS'

In [ ]:
# troubleshooting... who put a '/' in a student's name?! 
school = 'Blackman Middle School'
p = f'../YouScienceData/Schedules/{school}/{school}_all_students.csv'
df = pd.read_csv(p)
df.drop('Unnamed: 0', axis=1, inplace=True)

df.loc[df.name == 'JORDIN/ALBUS MCCLANAHAN'] # Comes from Original YouScience doc... Not found anywhere else. wtf?! 

In [ ]:
# test case student
test_case = school_all_df.iloc[5]
# Needed featuers: (1) student name, (2) Homeroom Teacher name, (3a-3d) Block Pathway title, 
# (4a-4d) Pathway Instructor, (5a-5d) Pathway room #, (6a-6d) Pathway zoom link
# (1), (2)
student_name, HR_teacher = test_case['name'], test_case['CY Teacher']
# (3a) 
pathway = test_case.B1
POS_info = planning_df.loc[planning_df.POS == pathway]
# 4a, 5a, 6a
POS_instructors = ' & '.join([POS_info['MS Teacher'].values[0], POS_info['HS Teacher'].values[0]])
POS_room_no = POS_info['MS Room #'].values[0]
POS_zoom_link = POS_info['Zoom Link'].values[0]

test_document = Document()
test_document.add_heading(f'Name: {student_name}', level = 1)
test_document.add_paragraph(f'Homeroom: {HR_teacher}')
test_document.add_heading(f'Pathway 1: {pathway}', level = 3)
test_document.add_paragraph(
    f'Sponsor/Instructors: {POS_instructors}\nRoom #: {POS_room_no}\nZoom: {POS_zoom_link}'
)

test_document.save('../test_schedule.docx')

In [ ]:
# Test version 2: adding for loop to get each pathway paragraph/heading
date = '8 February 2023'
test_case2 = school_all_df.iloc[10]
# (1), (2)
student_name, HR_teacher = test_case2['name'], test_case2['CY Teacher']
# Initializing document
test_document2 = Document()
section = test_document2.sections[0]
header = section.header
head_para = header.paragraphs[0]
head_para.text = f'{school}\tYouScience Career Fair\t{date}'
test_document2.add_heading(f'Name: {student_name}', level = 1)
test_document2.add_paragraph(f'Homeroom: {HR_teacher}')

for block in ['B1','B2','B3','B4']: 
    # get pathway's info
    # 3a
    pathway = test_case2[block]
    POS_info = planning_df.loc[planning_df.POS == pathway]
    # 4a, 5a, 6a
    POS_instructors = ' & '.join([POS_info['MS Teacher'].values[0], POS_info['HS Teacher'].values[0]])
    POS_room_no = POS_info['MS Room #'].values[0]
    POS_zoom_link = POS_info['Zoom Link'].values[0]

    # structure document
    test_document2.add_heading(f'Pathway {block[1]}: {pathway}', level=3)
    test_document2.add_paragraph(
        f'Sponsor/Instructors: {POS_instructors}\nRoom #: {POS_room_no}\nZoom: {POS_zoom_link}'
    )

test_document2.save('../test_schedule3.docx')